In [11]:
%load_ext autoreload
%autoreload 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import lib
import torch, numpy as np, pandas as pd, pytorch_lightning as pl, matplotlib.pyplot as plt
from torch import nn
from torchvision.transforms.functional import resize

In [13]:
opts = lib.get_default_opts()
opts['visualize_saliency'] = False

In [14]:
loaders = lib.get_loaders(opts)

In [23]:
batch = next(iter(loaders['valid']))

In [16]:
batch['image'].shape, batch['saliency'].shape

(torch.Size([16, 3, 384, 512]), torch.Size([16, 1, 384, 512]))

In [17]:
device = torch.device('cpu')
model = lib.Model(opts)

Model(
  (backbone): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivation(
  

In [ ]:
device = torch.device('cuda:0')
model.to(device)

In [24]:
pred = model(batch['image'].to(device), batch['saliency'].to(device))

In [25]:
((pred.cpu().squeeze() - batch['subj_mean'].squeeze()) ** 2).mean()

tensor(41.0377, grad_fn=<MeanBackward0>)

In [26]:
pred.squeeze()

tensor([64.6468, 51.4856, 51.9159, 76.9536, 74.2749, 37.0224, 72.3525, 59.1455,
        41.8018, 36.9963, 39.2661, 53.9092, 73.4232, 61.2551, 73.8985, 41.9476],
       device='cuda:0', grad_fn=<SqueezeBackward0>)

In [27]:
batch['subj_mean'].squeeze()

tensor([56.8623, 46.8728, 41.6500, 78.5340, 81.0735, 41.8119, 75.1801, 52.7185,
        50.2750, 38.3347, 52.3110, 52.0895, 79.8256, 64.1995, 69.6250, 47.7257])